In [1]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets('./mnist/data/', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
mnist.train.images.shape

(55000, 784)

In [6]:
X=tf.placeholder(tf.float32, [None,28,28,1])
Y=tf.placeholder(tf.float32, [None,10])

keep_prob=tf.placeholder(tf.float32)

W1=tf.Variable(tf.random_normal([3,3,1,32], stddev=.01))
L1=tf.nn.conv2d(X,W1,strides=[1,1,1,1], padding='SAME')
L1=tf.nn.relu(L1)
L1=tf.nn.max_pool(L1, ksize=[1,2,2,1],strides=[1,2,2,1], padding='SAME')

W2=tf.Variable(tf.random_normal([3,3,32,64], stddev=.01))
L2=tf.nn.conv2d(L1,W2,strides=[1,1,1,1], padding='SAME')
L2=tf.nn.relu(L2)
L2=tf.nn.max_pool(L2 , ksize=[1,2,2,1],strides=[1,2,2,1], padding='SAME')

W3=tf.Variable(tf.random_normal([7*7*64,256], stddev=.01))
L3=tf.reshape(L2,[-1,7*7*64])
L3=tf.matmul(L3,W3)
L3=tf.nn.relu(L3)
L3=tf.nn.dropout(L3,keep_prob)

W4=tf.Variable(tf.random_normal([256,10], stddev=.01))
model=tf.matmul(L3,W4)

cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y,logits=model))
optimizer=tf.train.AdamOptimizer(.001).minimize(cost)

In [8]:
init=tf.global_variables_initializer()
sess=tf.Session()
sess.run(init)

batch_size=100
total_batch=int(mnist.train.num_examples/batch_size)

for epoch in range(15):
    total_cost=0
    
    for i in range(total_batch):
        batch_xs,batch_ys=mnist.train.next_batch(batch_size)
        batch_xs=batch_xs.reshape(-1,28,28,1)
        
        _,cost_val=sess.run([optimizer, cost], feed_dict={X:batch_xs, Y:batch_ys, keep_prob:.7})
        
        total_cost += cost_val

    print('Epoch:%04d, Avg. cost %.3f' %(epoch+1,total_cost/batch_size))   

Epoch:0001, Avg. cost 1.936
Epoch:0002, Avg. cost 0.593
Epoch:0003, Avg. cost 0.426
Epoch:0004, Avg. cost 0.342
Epoch:0005, Avg. cost 0.274
Epoch:0006, Avg. cost 0.231
Epoch:0007, Avg. cost 0.199
Epoch:0008, Avg. cost 0.174
Epoch:0009, Avg. cost 0.149
Epoch:0010, Avg. cost 0.133
Epoch:0011, Avg. cost 0.119
Epoch:0012, Avg. cost 0.114
Epoch:0013, Avg. cost 0.096
Epoch:0014, Avg. cost 0.098
Epoch:0015, Avg. cost 0.081


In [9]:
is_correct=tf.equal(tf.argmax(Y,1), tf.argmax(model,1))
accuracy=tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('Accuracy:',sess.run(accuracy, feed_dict={X:mnist.test.images.reshape(-1,28,28,1),
                                               Y:mnist.test.labels,
                                               keep_prob:1}))

Accuracy: 0.9921


# 7.3 HighLevel API

In [10]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets('./mnist/data/', one_hot=True)

X=tf.placeholder(tf.float32, [None,28,28,1])
Y=tf.placeholder(tf.float32, [None,10])
is_training=tf.placeholder(tf.bool)

L1=tf.layers.conv2d(X,32,[3,3], activation=tf.nn.relu)
L1=tf.layers.max_pooling2d(L1,[2,2],[2,2])#ksize, strides
L1=tf.layers.dropout(L1,.7,is_training)

L2=tf.layers.conv2d(L1,64,[3,3], activation=tf.nn.relu)
L2=tf.layers.max_pooling2d(L2,[2,2],[2,2])#ksize, strides
L2=tf.layers.dropout(L2,.7,is_training)

L3=tf.contrib.layers.flatten(L2)
L3=tf.layers.dense(L3,256,activation=tf.nn.relu)
L3=tf.layers.dropout(L3,.5,is_training)

model=tf.layers.dense(L3,10,activation=None)

cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))
optimizer=tf.train.AdamOptimizer(.001).minimize(cost)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [13]:
init=tf.global_variables_initializer()
sess=tf.Session()
sess.run(init)

batch_size=100
total_batch=int(mnist.train.num_examples/ batch_size)

for epoch in range(15):
    total_cost=0
    
    for i in range(total_batch):
        batch_xs, batch_ys=mnist.train.next_batch(batch_size)
        batch_xs=batch_xs.reshape(-1,28,28,1)
        _,cost_val=sess.run([optimizer, cost], feed_dict={X:batch_xs, Y:batch_ys, is_training:True})
        total_cost += cost_val
        
    print('Epoch:', epoch+1,'\t Avg.cost:',total_cost/batch_size)

Epoch: 1 	 Avg.cost: 0.973786380593665
Epoch: 2 	 Avg.cost: 0.28401529691182076
Epoch: 3 	 Avg.cost: 0.19159172421204856
Epoch: 4 	 Avg.cost: 0.13472390236740467
Epoch: 5 	 Avg.cost: 0.09729414122761228
Epoch: 6 	 Avg.cost: 0.08477792779478477
Epoch: 7 	 Avg.cost: 0.060201402711536504
Epoch: 8 	 Avg.cost: 0.05675248563857167
Epoch: 9 	 Avg.cost: 0.03720425536866969
Epoch: 10 	 Avg.cost: 0.04421077268907538
Epoch: 11 	 Avg.cost: 0.034669768399226086
Epoch: 12 	 Avg.cost: 0.023649986468390127
Epoch: 13 	 Avg.cost: 0.020211200998196545
Epoch: 14 	 Avg.cost: 0.035320131949974895
Epoch: 15 	 Avg.cost: 0.017136596183793245


In [14]:
is_correct=tf.equal(tf.argmax(Y,1), tf.argmax(model,1))
accuracy=tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('Accuracy:',sess.run(accuracy, feed_dict={X:mnist.test.images.reshape(-1,28,28,1),
                                               Y:mnist.test.labels,
                                               is_training:False}))

Accuracy: 0.9902
